In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"                                    

In [109]:
import pandas as pd 
import numpy as np

# Importing database of food items

In [141]:
df=pd.read_csv('palmshore-Menu.csv')
df.columns = ['food_id','title','price', 'num_orders', 'category', 'avg_rating', 'num_rating', 'tags']
df.head()

,food_id,title,price,num_orders,category,avg_rating,num_rating,tags
0,1,Tandoori Chicken,150,164,non-veg,4.1,62,non-veg tandoori chicken
1,2,Special Kizhi Parotta,260,62,non-veg,3.5,31,non-veg parotta chicken
2,3,Basa Fish Tawa Grill,360,28,non-veg,4.0,14,non-veg fish grill
3,4,Chicken Fried Rice,230,67,non-veg,4.3,33,non-veg chicken rice
4,5,Tawa Pepper Bbq Chicken,290,10,non-veg,4.8,5,non-veg chicken bbq


In [121]:
# mean of average ratings of all items
avg_rmean=df['avg_rating'].mean()

# the minimum number of votes required to appear in recommendation list, i.e, 60th percentile among 'num_rating'
min_votes= df['num_rating'].quantile(0.9)

# items that qualify the criteria of minimum num of votes
qualified_items=df.copy().loc[df['num_rating']>=min_votes]

# Calculation of weighted rating based on the IMDB formula
def weighted_rating(x, min_votes=min_votes, avg_rmean=avg_rmean):
    v = x['num_rating']
    R = x['avg_rating']
    return (v/(v+min_votes) * R) + (min_votes/(min_votes+v) * avg_rmean)

# Applying weighted_rating to qualified items
qualified_items['score']=qualified_items.apply(weighted_rating,axis=1)

#shortlisting the top rated items and popular items
top_rated_items=qualified_items.sort_values('score',ascending=False)
popular_items=df.sort_values('num_orders',ascending=False)

# Get Top Rated Items

In [130]:
top_rated_items[['title']][:5].style.hide_index()

title
Mix Noodles Non Veg
Chilly Prawn Dry
Chicken Fried Rice
Tandoori Chicken
Paya Pepper Masala


# Get Popular Items

In [131]:
popular_items[['title']][:5].style.hide_index()

title
Tandoori Chicken
Tandoori Paratha Combo
Chicken Fried Rice
Singapore Tiger Prawn
Special Kizhi Parotta


In [153]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(stop_words='english')

count_matrix = count.fit_transform(df['tags'])

from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(count_matrix, count_matrix)

indices_from_title = pd.Series(df.index, index=df['title'])
indices_from_food_id = pd.Series(df.index, index=df['food_id'])


# Get Recommendations based on title

In [154]:
def get_recommendations(title="", cosine_sim=cosine_sim, idx=-1):
    if idx == -1 and title != "":
        idx = indices_from_title[title]
    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    sim_scores = sim_scores[:len(sim_scores)]

    food_indices = [i[0] for i in sim_scores]
    return food_indices


In [163]:
df.loc[get_recommendations(title='Gobi 65')][:5][['title']].style.hide_index()

title
Gobi Manchurian Gravy
Gobi 65
Gobi Chilly Dry
Gobi Manchurian Dry
Gobi Chilly Gravy
